In [1]:
# Cell 1: Import necessary libraries and create Spark session
import findspark
findspark.init()
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import functions as F
from pyspark.ml.feature import Imputer
from mlxtend.regressor import StackingCVRegressor


# Create Spark session
spark = SparkSession.builder.appName("HousePricePrediction").getOrCreate()


# Cell 2: Read the data
df_train = spark.read.csv("data/train.csv", header=True, inferSchema=True)
df_test = spark.read.csv("data/test.csv", header=True, inferSchema=True)


# Cell 3: Drop unnecessary columns and handle missing values for "MSZoning" column
cols_to_drop = ['FireplaceQu', 'Fence', 'Alley', 'MiscFeature', 'PoolQC']
df_train_cleaned = df_train.drop(*cols_to_drop)
df_test_cleaned = df_test.drop(*cols_to_drop)

mszoning_mode = df_train_cleaned.select("MSZoning").groupBy("MSZoning").count().orderBy(F.col("count").desc()).first()["MSZoning"]
df_train_cleaned = df_train_cleaned.na.fill({"MSZoning": mszoning_mode})
df_test_cleaned = df_test_cleaned.na.fill({"MSZoning": mszoning_mode})


# Cell 4: Handle missing values for both categorical and numerical features

# Handle missing values for categorical features
categorical_cols = [col_name for col_name, dtype in df_train_cleaned.dtypes if dtype == "string"]
for col in categorical_cols:
    mode_value = df_train_cleaned.select(col).groupBy(col).count().orderBy(F.col("count").desc()).first()[col]
    df_train_cleaned = df_train_cleaned.na.fill({col: mode_value})
    df_test_cleaned = df_test_cleaned.na.fill({col: mode_value})

# Handle missing values for numerical features
numerical_cols = [col_name for col_name, dtype in df_train_cleaned.dtypes if dtype != "string" and col_name != "Id" and col_name != "SalePrice"]
for col in numerical_cols:
    df_train_cleaned = df_train_cleaned.withColumn(col, F.col(col).cast("double"))
    df_test_cleaned = df_test_cleaned.withColumn(col, F.col(col).cast("double"))

imputer = Imputer(inputCols=numerical_cols, outputCols=[f"{col}_imputed" for col in numerical_cols])
imputer_model = imputer.fit(df_train_cleaned)
df_train_cleaned = imputer_model.transform(df_train_cleaned)
df_test_cleaned = imputer_model.transform(df_test_cleaned)


# Cell 5: Convert columns to the correct data types (after filling missing values for categorical columns)
for col in df_train_cleaned.columns:
    if col != "Id" and col != "SalePrice":
        df_train_cleaned = df_train_cleaned.withColumn(col, F.col(col).cast("double"))
        df_test_cleaned = df_test_cleaned.withColumn(col, F.col(col).cast("double"))


# Cell 6: Data Cleaning and Preprocessing using pandas and scikit-learn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
sns.set(style="darkgrid", font_scale=1.5)
pd.set_option("display.max.columns", None)

# Drop columns with a high percentage of missing values
missing_threshold = 0.8
cols_to_drop = [col for col in df_train.columns if (df_train.select(col).na.drop().count() / df_train.count()) < missing_threshold]
df_train_cleaned = df_train.drop(*cols_to_drop)
df_test_cleaned = df_test.drop(*cols_to_drop)

# Fill missing numerical values with the mean
numerical_cols_with_missing = [col_name for col_name, dtype in df_train_cleaned.dtypes if dtype != "string" and col_name != "Id" and col_name != "SalePrice" and df_train_cleaned.select(col_name).na.drop().count() != df_train_cleaned.count()]
for col in numerical_cols_with_missing:
    mean_value = df_train_cleaned.select(col).agg(avg(col)).collect()[0][0]
    df_train_cleaned = df_train_cleaned.fillna(mean_value, subset=[col])
    df_test_cleaned = df_test_cleaned.fillna(mean_value, subset=[col])

# Fill missing categorical values with the most common value
categorical_cols_with_missing = [col_name for col_name, dtype in df_train_cleaned.dtypes if dtype == "string" and df_train_cleaned.select(col_name).na.drop().count() != df_train_cleaned.count()]
for col in categorical_cols_with_missing:
    most_common_value = df_train_cleaned.groupBy(col).count().orderBy("count", ascending=False).first()[0]
    df_train_cleaned = df_train_cleaned.fillna(most_common_value, subset=[col])
    df_test_cleaned = df_test_cleaned.fillna(most_common_value, subset=[col])

# Convert categorical columns to string type
for col_name, dtype in df_train_cleaned.dtypes:
    if dtype == "string":
        df_train_cleaned = df_train_cleaned.withColumn(col_name, df_train_cleaned[col_name].cast("string"))
        df_test_cleaned = df_test_cleaned.withColumn(col_name, df_test_cleaned[col_name].cast("string"))


# Cell 7: Feature Engineering and Transformation

# Convert numerical columns to numeric data type
for col_name in ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath", "GarageCars", "GarageArea"]:
    df_train_cleaned = df_train_cleaned.withColumn(col_name, col(col_name).cast("double"))
    df_test_cleaned = df_test_cleaned.withColumn(col_name, col(col_name).cast("double"))

# Feature engineering
categorical_cols = [col_name for col_name, dtype in df_train_cleaned.dtypes if dtype == "string"]
numerical_cols = [col_name for col_name, dtype in df_train_cleaned.dtypes if dtype != "string" and col_name != "Id" and col_name != "SalePrice"]

# Assemble the features
assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
df_train_assembled = assembler.transform(df_train_cleaned)
df_test_assembled = assembler.transform(df_test_cleaned)

# Scale the features
scaler = StandardScaler(inputCol="numerical_features", outputCol="scaled_numerical_features", withMean=True, withStd=True)
scaler_model = scaler.fit(df_train_assembled)
df_train_scaled = scaler_model.transform(df_train_assembled)
df_test_scaled = scaler_model.transform(df_test_assembled)

# One-hot encoding for categorical features
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid='keep') for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol=f"{indexer.getOutputCol()}_encoded") for indexer in indexers]

pipeline = Pipeline(stages=indexers + encoders)
df_train_encoded = pipeline.fit(df_train_scaled).transform(df_train_scaled)
df_test_encoded = pipeline.fit(df_test_scaled).transform(df_test_scaled)

# Assemble all features
assembler = VectorAssembler(inputCols=[f"{col}_encoded" for col in categorical_cols] + ["scaled_numerical_features"],
                            outputCol="features")
df_train_final = assembler.transform(df_train_encoded)
df_test_final = assembler.transform(df_test_encoded)

# Select the final features for modeling
df_train_final = df_train_final.select("Id", "features", "SalePrice")
df_test_final = df_test_final.select("Id", "features")


# Cell 8: Model Training and Evaluation using Cross-Validation
# Split data into training and validation sets
train_data, validation_data = df_train_final.randomSplit([0.8, 0.2], seed=42)

# Initialize Linear Regression model
lr = LinearRegression(featuresCol='features', labelCol='SalePrice', maxIter=100, regParam=0.1)

# Set up the parameter grid for hyperparameter tuning
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.1, 0.5]).build()

# Initialize CrossValidator
evaluator = RegressionEvaluator(labelCol="SalePrice", predictionCol="prediction", metricName="rmse")
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Train the model using CrossValidator
cvModel = cv.fit(train_data)

# Make predictions on the validation set
validation_predictions = cvModel.transform(validation_data)

# Evaluate the model on the validation set
rmse = evaluator.evaluate(validation_predictions)
print(f"Root Mean Squared Error (RMSE) on validation data: {rmse:.2f}")


# Cell 9: Model Prediction on Test Data and Save Results
# Make predictions on the test set
test_predictions = cvModel.transform(df_test_final)

# Select the necessary columns for the final result
final_result = test_predictions.select("Id", "prediction").withColumnRenamed("prediction", "SalePrice")

# Save the predictions to a CSV file
final_result.coalesce(1).write.csv("predictions.csv", header=True, mode="overwrite")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/07 10:53:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
